In [1]:
import numpy as np
import chainer
from chainer import cuda, Function, gradient_check, Variable
from chainer import optimizers, serializers, utils
from chainer import Link, Chain, ChainList
import chainer.functions as F
import chainer.links as L

In [2]:
# Set data

from sklearn import datasets
iris = datasets.load_iris()
X = iris.data.astype(np.float32)
Y = iris.target
N = Y.size
Y2 = np.zeros(3 * N).reshape(N,3).astype(np.float32)
for i in range(N):
    Y2[i,Y[i]] = 1.0

In [3]:
index = np.arange(N)
xtrain = X[index[index % 2 != 0],:]
ytrain = Y2[index[index % 2 != 0],:]
xtest = X[index[index % 2 == 0],:]
yans = Y[index[index % 2 == 0]]

In [4]:
# Define model

class IrisChain(Chain):
    def __init__(self):
        super(IrisChain, self).__init__(
            l1=L.Linear(4,6),
            l2=L.Linear(6,3),
        )

    def __call__(self,x,y):
        return F.mean_squared_error(self.fwd(x), y)

    def fwd(self,x):
         h1 = F.sigmoid(self.l1(x))
         h2 = self.l2(h1)
         return h2

In [5]:
# Initialize model

model = IrisChain()
optimizer = optimizers.SGD()
optimizer.setup(model)

In [6]:
# Learn

n = 75
bs = 25
for j in range(5000):
    sffindx = np.random.permutation(n)
    for i in range(0, n, bs):
        x = Variable(xtrain[sffindx[i:(i+bs) if (i+bs) < n else n]])
        y = Variable(ytrain[sffindx[i:(i+bs) if (i+bs) < n else n]])
        model.zerograds()
        loss = model(x,y)
        loss.backward()
        optimizer.update()

In [7]:
# Test

xt = Variable(xtest, volatile='on')
yy = model.fwd(xt)

ans = yy.data
nrow, ncol = ans.shape
ok = 0
for i in range(nrow):
    cls = np.argmax(ans[i,:])
    print( ans[i,:], cls)
    if cls == yans[i]:
        ok += 1

print( ok, "/", nrow, " = ", (ok * 1.0)/nrow)

[ 1.00325894  0.00880763 -0.01487979] 0
[ 0.97348583  0.03704482 -0.02449787] 0
[  1.00909436e+00   8.91506672e-04  -1.08515918e-02] 0
[ 0.97545493  0.03668633 -0.01846156] 0
[ 0.90680301  0.12108111 -0.04760715] 0
[ 1.01798427 -0.00362083 -0.01009277] 0
[ 0.94223142  0.08486298 -0.04312506] 0
[ 1.06100094 -0.06446362  0.00930771] 0
[ 1.04445577 -0.04488614  0.00526303] 0
[ 1.01138246  0.0141772  -0.01521128] 0
[ 0.96258724  0.08004391 -0.03991735] 0
[ 1.02901614 -0.04012898 -0.00183922] 0
[ 0.88624811  0.19014177 -0.06692562] 0
[ 0.9581089   0.07285154 -0.03103879] 0
[ 0.99713111  0.01704663 -0.01910383] 0
[ 0.91478133  0.1316247  -0.05322701] 0
[ 1.04352105 -0.03576836  0.00338075] 0
[ 0.94196844  0.09265327 -0.04468936] 0
[ 1.02139688 -0.01610789 -0.00951508] 0
[ 0.93865538  0.07525176 -0.03454053] 0
[ 1.00886345 -0.00431272 -0.00976348] 0
[ 0.96206248  0.04670873 -0.02427256] 0
[ 0.95422626  0.0961183  -0.03493798] 0
[ 1.00718153  0.01308137 -0.01223609] 0
[  1.01543653e+00  -8.173

In [9]:
test_index = 68
xa = Variable(np.array([xtest[test_index]]).astype(np.float32()), volatile='on')
ya = model.fwd(xa)

print(ya.data)
print(yans[test_index])

[[-0.11401099  0.07830894  1.01588249]]
2
